In [2]:
import mxnet as mx
# construct a cnn
net = mx.sym.Variable('data')
net = mx.sym.Convolution(data = net,kernel=(2,2),stride=(2,2),pad=(1,1),num_filter=5,name='conv1')
net = mx.sym.Activation(data = net,name='act_1',act_type='relu')
net = mx.sym.Flatten(net)
net1 = mx.sym.FullyConnected(data=net,num_hidden=2,name='fc1')
output = mx.sym.SoftmaxOutput(data=net1,name='softmax1')


In [3]:
mod = mx.mod.Module(symbol=output,
                    context = mx.gpu(),
                    data_names=['data'],
                    label_names=['softmax1_label'])

In [4]:
from gen_input import gen_input
test_data_iter = gen_input('./input/test_data','test_image')
test_data_iter.reset()
test_data = test_data_iter.next()
test_data_features = test_data.data[0]
test_data_label=test_data.label[0]
#这里可以继承Image.ImageIter，将其改造成mx.io.DataIter这个接口

In [5]:
#这个数据还不太好使，我换别的数据试试
train_data_iter = gen_input('./input/train_data','train_image')
train_data_iter.reset()
train_data = train_data_iter.next()
train_data_features = train_data.data[0]
train_data_label=train_data.label[0]
#这里可以继承Image.ImageIter，将其改造成mx.io.DataIter这个接口

In [6]:
#这个数据还不太好使，我换别的数据试试
train_data_iter = gen_input('./input/train_data','train_image')
train_data_iter.reset()
train_data = train_data_iter.next()
train_data_features = train_data.data[0]
train_data_label=train_data.label[0]
#这里可以继承Image.ImageIter，将其改造成mx.io.DataIter这个接口

In [7]:
#重新定义iter
class Multi_iterator(mx.io.DataIter):
    '''multi label  iterator'''

    def __init__(self, data_iter):
        super(Multi_iterator, self).__init__()
        self.data_iter = data_iter
        self.batch_size = self.data_iter.batch_size

    @property
    def provide_data(self):
        return self.data_iter.provide_data

    @property
    def provide_label(self):
        provide_label = self.data_iter.provide_label[0]
        # Different labels should be used here for actual application
        #return [('softmax1_label', provide_label[1]), \
        #        ('softmax2_label', provide_label[1])]
        return [('softmax1_label',(provide_label[1][0],))]

    def hard_reset(self):
        self.data_iter.hard_reset()

    def reset(self):
        self.data_iter.reset()

    def next(self):
        batch = self.data_iter.next()
        label = batch.label[0]

        return mx.io.DataBatch(data=batch.data, label=[label[:,0]], \
                pad=batch.pad, index=batch.index)

In [8]:
train_iter = Multi_iterator(train_data_iter)
test_iter = Multi_iterator(test_data_iter)

In [9]:
mod.bind(data_shapes=train_iter.provide_data,label_shapes=train_iter.provide_label)
mod.init_params(initializer=mx.init.Uniform(scale=0.1))
mod.init_optimizer(optimizer='sgd',optimizer_params=(('learning_rate',0.1),))

In [10]:
metric = mx.metric.create('acc')
num_epoch = 10
for epoch in range(num_epoch):
    train_iter.reset()
    metric.reset()
    for batch in train_iter:
        mod.forward(batch,is_train=True)
        mod.update_metric(metric,batch.label)
        mod.backward()
        mod.update()
    print('epoch %d train acc: %s' % (epoch,metric.get()))

epoch 0 train acc: ('accuracy', 0.50242477968399646)
epoch 1 train acc: ('accuracy', 0.49757522031600354)
epoch 2 train acc: ('accuracy', 0.50242477968399646)
epoch 3 train acc: ('accuracy', 0.50242477968399646)
epoch 4 train acc: ('accuracy', 0.50242477968399646)
epoch 5 train acc: ('accuracy', 0.50242477968399646)
epoch 6 train acc: ('accuracy', 0.50242477968399646)
epoch 7 train acc: ('accuracy', 0.50242477968399646)
epoch 8 train acc: ('accuracy', 0.50242477968399646)
epoch 9 train acc: ('accuracy', 0.50242477968399646)
